In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix, hstack

In [2]:
PATH = '../../data/'
### PATH on CMC AI
PATH = '/home/kai/data/kaggle/toxic/jc/data/'
train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

train_sentence = train['comment_text_cleaned']
test_sentence = test['comment_text_cleaned']

text = pd.concat([train_sentence, test_sentence])

print(train.shape)
print(test.shape)

(159571, 30)
(153164, 24)


In [3]:
# CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, max_df=1.0,\
#                 min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>)

# TfidfVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None,\
#                 vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>, norm=’l2’, use_idf=True,\
#                 smooth_idf=True, sublinear_tf=False)

print('getting tf count')
phrase_vectorizer = CountVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    token_pattern=r'\w{1,}')
char_vectorizer = CountVectorizer(ngram_range=(1,5), 
                                  strip_accents='unicode', 
                                  max_features=200000, 
                                  analyzer='char')



print('fitting char')
char_vectorizer.fit(text.values)
print('fitting phrase')
phrase_vectorizer.fit(text.values)
print('transforming train char')
train_char_count = char_vectorizer.transform(train_sentence.values)
print('transforming train phrase')
train_phrase_count = phrase_vectorizer.transform(train_sentence.values)
print('transforming test char')
test_char_count = char_vectorizer.transform(test_sentence.values)
print('transforming test phrase')
test_phrase_count = phrase_vectorizer.transform(test_sentence.values)





train_tf = hstack((train_char_count, train_phrase_count), format='csr')
test_tf = hstack((test_char_count, test_phrase_count), format='csr')

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train_tf

getting tf count
fitting char
fitting phrase
transforming train char
transforming train phrase
transforming test char
transforming test phrase


<159571x300000 sparse matrix of type '<class 'numpy.int64'>'
	with 153144273 stored elements in Compressed Sparse Row format>

In [7]:
def corr2_coeff(A,B):
    # this function returns corr matrix of n*a matrix A and n*b matrix B
    
    matrix_corr = np.zeros((A.shape[1], B.shape[1]))

    for j in range(A.shape[1]):
        a = A[:,j].todense()

        for i in range(B.shape[1]):
            b = B[:,i].todense()
            
            numerator = np.dot((a - a.mean()).T , (b - b.mean()))/a.shape[0]

            denomitor = np.std(a) * np.std(b)

            matrix_corr[j,i] = numerator / denomitor
            
        file = open('tf_printing.txt','a')
        file.write(str(j) +'\n')
        file.close()
    
        print(j)
    
    # Finally get corr coeff
    return matrix_corr

def get_corr_reg_model(label_cols, train_tf, train, test_tf):
    
    preds = np.zeros((test.shape[0], len(label_cols)))
    train_preds = np.zeros((train.shape[0], len(label_cols)))
    
    l = csr_matrix(train[label_cols].values)
    cor = pd.read_csv(PATH + 'corr.csv',header=None)
    cor = cor.fillna(0).values
    
    for i, j in enumerate(label_cols):
        print('fit', j)
        y = train[j].values
        r = cor[:,i]
        
        model = LogisticRegression()
        print(r.shape)
        print(train_tf.shape)
        x_cor = train_tf.multiply(r).tocsr()
        model.fit(x_cor, y)
        preds[:, i] = model.predict_proba(test_tf.multiply(r))[:, 1]
        train_preds[:, i] = model.predict_proba(x_cor)[:, 1]
        print('accuracy is {}'.format(roc_auc_score(y, train_preds[:, i])))
        
    return preds, train_preds

def save(model_name, y_test, label_cols, path, is_train=False):
    if is_train:
        submission = pd.read_csv(path + 'sample_train.csv')
        file_name = 'train_' + model_name
    else:
        submission = pd.read_csv(path + 'sample_submission.csv')
        file_name = model_name
    submission[label_cols] = y_test
    submission.to_csv(path + '/' + file_name + '.csv', index=False)
    
print('done')

done


In [5]:
import os
os.environ['MKL_NUM_THREADS']='6'
os.environ['NUMEXPR_NUM_THREADS']='6'
os.environ['OMP_NUM_THREADS']='6'

print('predicting')
y_test, y_train = get_corr_reg_model(label_cols, train_tf, train, test_tf)
print('total score is {}'.format(roc_auc_score(train[label_cols], y_train)))



predicting
fit toxic
(300000,)
(159571, 300000)
accuracy is 0.916282195943934
fit severe_toxic
(300000,)
(159571, 300000)
accuracy is 0.9527648023357542
fit obscene
(300000,)
(159571, 300000)
accuracy is 0.9271203827609978
fit threat
(300000,)
(159571, 300000)
accuracy is 0.8535640267986722
fit insult
(300000,)
(159571, 300000)
accuracy is 0.9141844886950515
fit identity_hate
(300000,)
(159571, 300000)
accuracy is 0.8686267430277204
total score is 0.9054237732603551
saving files


FileNotFoundError: [Errno 2] No such file or directory: '/home/kai/data/kaggle/toxic/jc/data/tf_cor_log_reg/tf_cor_log_reg.csv'

In [12]:
cor = pd.read_csv(PATH + 'corr.csv', header=None)

In [8]:
print('saving files')
model_name = 'tf_cor_log_reg'
save(model_name, y_test, label_cols, PATH)
save(model_name, y_train, label_cols, PATH, True)

print('done')

saving files
done
